# Strands - Multiple MCP

- `Strands Agent`에 대해 LLM에게 물어 본다면, 아직은 최신 프레임워크인 Strands에 대해 잘못된 답변을 합니다.
- 이러한 이유로, 
    - MCP 1 : 웹 검색을 통해서 `Strands Agent`에 대한 정확한 데이터를 검색하도록 합니다. 
    - 검색 한 결과를 바탕으로 LLM은 상세 보고서를 작성 합니다. 
    - MCP 2 : 작성한 보고서를 `Notion` Page로 작성 합니다. 

In [33]:
import boto3
from strands.models import BedrockModel

bedrock_model = BedrockModel(
    # model_id="anthropic.claude-sonnet-4-20250514-v1:0",
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    region_name="us-west-2",
    temperature=0.3,
)

In [34]:

from mcp import stdio_client, StdioServerParameters
from mcp.client.streamable_http import streamablehttp_client
from strands.agent.conversation_manager import SlidingWindowConversationManager

from strands import Agent
from strands.tools.mcp import MCPClient

import os
from dotenv import load_dotenv
load_dotenv()

True

In [35]:
NOTION_API_KEY = os.environ.get("NOTION_API_KEY")
str_notion_mcp = "{\"Authorization\": \"Bearer " + NOTION_API_KEY + "\", \"Notion-Version\": \"2022-06-28\" }"
EXA_API_KEY = os.environ.get("EXA_API_KEY")
TAVILY_API_KEY = os.environ.get("TAVILY_API_KEY")
GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN")

In [36]:
# 환경 변수 및 MCP 서버 경로 세팅
MCP_SERVERS = {
    "thinking" : {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-sequential-thinking"],},  # thinking
    "duckduckgo" : {"command": "npx", "args": ["-y", "@pinkpixel/web-scout-mcp"],},  # DuckDuckGo
    "tavily" : {"url" : f"https://mcp.tavily.com/mcp/?tavilyApiKey={TAVILY_API_KEY}"},  # Tavily
    "notion" : {"command": "npx", "args": ["-y", "@notionhq/notion-mcp-server"], "env": {"OPENAPI_MCP_HEADERS": str_notion_mcp, "NOTION_PARENT_ID":"238e297222b8805c8cf1f0ff6ffda252"}},
    "github" : {"command": "npx", "args":["-y", "@modelcontextprotocol/server-github"], "env": { "GITHUB_TOKEN": GITHUB_TOKEN }},  # github
}


MCP_SERVER_HTTP = {"Notion":  "https://mcp.notion.com/mcp",}


conversation_manager = SlidingWindowConversationManager(window_size=7)

In [37]:

# prompt = "먼저, Strand Agent에 대해 인터넷 검색하고, 요약 해. 상세 보고서를 작성 해. 작성된 결과는 Notion의 새 페이지에 기록 해. 부모페이지 아이디는 238e297222b8805c8cf1f0ff6ffda252 야"
# prompt = " Agent AI에 대해서, 간략히 요약하고 Notion 최상단 페이지를 하나 생성하여 요약 내용을 기록 해"
# prompt = "최신 AI Agent에 뉴스 3개만 검색하고, 요약해. 최종적으로 결과 보고서를 Notion에 작성해. 부모 페이지 아이디는 '238e297222b8805c8cf1f0ff6ffda252' 야. "
prompt = "Strands Agent를 상세 보고서를 작성해. Github와 웹검색을 사용해서 자료를 수집하고 요약 보고서를 작성 해."

# config_duckduckgo = MCP_SERVERS["duckduckgo"]
# print(config_duckduckgo['args'])
# mcp_duckduckgo = MCPClient(lambda: stdio_client(StdioServerParameters(command=config_duckduckgo['command'], args=config_duckduckgo['args'], env=config_duckduckgo.get("env",None))))
# config_notion = MCP_SERVERS["notion"]
# print(config_notion['args'])
# mcp_notion = MCPClient(lambda: stdio_client(StdioServerParameters(command=config_notion['command'], args=config_notion['args'], env=config_notion.get("env",None))))


config_tavily = MCP_SERVERS["tavily"]
mcp_tavily = MCPClient(lambda: streamablehttp_client(config_tavily['url']) )

config_github = MCP_SERVERS["github"]
mcp_github = MCPClient(lambda: stdio_client(StdioServerParameters(command=config_github['command'], args=config_github['args'], env=config_github.get("env",None))))

# with mcp_duckduckgo, mcp_notion:
with mcp_tavily, mcp_github:
    # tools_0 = mcp_duckduckgo.list_tools_sync()
    tools_0 = mcp_tavily.list_tools_sync()
    tools_1 = mcp_github.list_tools_sync()
    all_tools = tools_0 + tools_1 
    
    # agent = Agent(tools=all_tools, conversation_manager=conversation_manager)
    agent = Agent(
        model=bedrock_model, # Throttling 때문에 모델 변경
        tools=all_tools, 
        conversation_manager=conversation_manager
    ) 
    results = agent(prompt)
    
    
    

Strands Agent에 대한 상세 보고서를 작성하기 위해 먼저 관련 정보를 수집하겠습니다. GitHub와 웹 검색을 통해 자료를 수집하겠습니다.

먼저 "Strands Agent"에 대한 웹 검색을 진행하겠습니다:
Tool #1: tavily_search
GitHub에서 Strands Agent 관련 정보를 검색해보겠습니다:
Tool #2: search_repositories
공식 AWS Strands Agents 저장소를 찾아보겠습니다:
Tool #3: search_repositories
다른 키워드로 검색해보겠습니다:
Tool #4: search_repositories
AWS 공식 문서에서 Strands Agent에 대한 정보를 찾아보겠습니다:
Tool #5: tavily_search
공식 문서 URL을 찾아보겠습니다:
Tool #6: tavily_search
공식 GitHub 저장소를 찾아보겠습니다:
Tool #7: tavily_search
공식 웹사이트를 확인해보겠습니다:
Tool #8: tavily_extract
GitHub에서 더 많은 정보를 찾아보겠습니다:
Tool #9: tavily_search
더 많은 정보를 수집하기 위해 Strands Agents의 사용 예제를 검색해보겠습니다:
Tool #10: tavily_search
공식 문서 페이지를 확인해보겠습니다:
Tool #11: tavily_extract
이제 수집한 정보를 바탕으로 Strands Agent에 대한 상세 보고서를 작성하겠습니다:

# Strands Agent 상세 보고서

## 1. 개요

AWS Strands Agents는 인공지능 에이전트를 쉽게 구축할 수 있도록 설계된 오픈소스 소프트웨어 개발 키트(SDK)입니다. 이 SDK는 모델 중심 접근 방식을 채택하여 대규모 언어 모델(LLM)의 추론 및 계획 능력을 활용해 최소한의 코드로 지능형 작업 특화 AI 에이전트를 구축하고 배포할 수 있게 합니다. 2025년 5월에 처음 공개 프리뷰로 출시되었으며, 2025년 7월에 버전

In [38]:
for i, m in enumerate(agent.messages):
    print(i,m)

0 {'role': 'user', 'content': [{'text': 'Strands Agent를 상세 보고서를 작성해. Github와 웹검색을 사용해서 자료를 수집하고 요약 보고서를 작성 해.'}]}
1 {'role': 'assistant', 'content': [{'text': 'Strands Agent에 대한 상세 보고서를 작성하기 위해 먼저 관련 정보를 수집하겠습니다. GitHub와 웹 검색을 통해 자료를 수집하겠습니다.\n\n먼저 "Strands Agent"에 대한 웹 검색을 진행하겠습니다:'}, {'toolUse': {'toolUseId': 'tooluse_rKGIvvZyTFmh9rXsyesejA', 'name': 'tavily_search', 'input': {'query': 'Strands Agent AI technology features', 'search_depth': 'advanced'}}}]}
2 {'role': 'user', 'content': [{'toolResult': {'status': 'success', 'toolUseId': 'tooluse_rKGIvvZyTFmh9rXsyesejA', 'content': [{'text': '{\n  "query": "Strands Agent AI technology features",\n  "follow_up_questions": null,\n  "answer": null,\n  "images": [],\n  "results": [\n    {\n      "url": "https://dev.to/aws-builders/aws-strands-agents-a-model-driven-approach-to-building-ai-agents-in-just-a-few-lines-of-code-1l77",\n      "title": "AWS Strands Agents: A model-driven approach to building AI agents ...",\n      "content": "_AWS

# 예제2

In [ ]:
stdio_mcp_web = MCPClient(lambda: stdio_client(
    StdioServerParameters(command="npx", args=["-y", "@pinkpixel/web-scout-mcp"])
))

stdio_mcp_thinking = MCPClient(lambda: stdio_client(
    StdioServerParameters(command="npx", args=["-y", "@modelcontextprotocol/server-sequential-thinking"])
))



with stdio_mcp_web:
    tools_web = stdio_mcp_web.list_tools_sync() # get tools from MCP
    
    with stdio_mcp_thinking:
        tools_thinking = stdio_mcp_thinking.list_tools_sync() # get tools from MCP
        
        tools = tools_web + tools_thinking
        agent = Agent(tools=tools)
        results = agent("최신 AI Agent에 대한 내용을 탐색/분석하고, Strand Agent의 특징과 장단점, 사용법 등 상세보고서를 작성해 줘.")

안녕하세요! 최신 AI Agent와 특히 Strand Agent에 대한 상세 보고서를 작성해 드리겠습니다. 먼저 관련 정보를 검색하고 분석해 보겠습니다.
Tool #1: DuckDuckGoWebSearch
제공된 검색 결과에 Strand Agents에 관한 더 자세한 정보를 확인하기 위해 추가 검색을 진행하겠습니다.
Tool #2: DuckDuckGoWebSearch
더 자세한 기술적 정보와 사용법을 알아보기 위해 몇 가지 페이지의 내용을 추출하겠습니다.
Tool #3: UrlContentExtractor

Tool #4: UrlContentExtractor

Tool #5: UrlContentExtractor

Tool #6: UrlContentExtractor

Tool #7: sequentialthinking

Tool #8: sequentialthinking

Tool #9: sequentialthinking

Tool #10: sequentialthinking

Tool #11: sequentialthinking

Tool #12: sequentialthinking

Tool #13: sequentialthinking

Tool #14: sequentialthinking
# 최신 AI Agent와 Strand Agents 상세 보고서

## 1. AI 에이전트(Agent)의 개념과 배경

AI 에이전트는 사용자의 목표를 달성하기 위해 데이터를 분석하고, 스스로 판단하며, 실제 행동을 수행하는 자율적 인공지능 시스템입니다. 기존의 생성형 AI가 콘텐츠 생성에 집중했다면, AI 에이전트는 문제 해결과 작업 실행까지 포괄합니다. 

**핵심 특징:**
- **자율성**: 인간의 개입 없이도 작업을 완료할 수 있는 능력
- **추론 및 계획 능력**: 복잡한 문제에 대한 단계별 접근 가능
- **도구 사용 능력**: 다양한 외부 도구와 API를 활용하여 작업 수행

최근 몇 년간 LLM(대규모 언어 모델)의 발전으로 인해 AI 에이전트 기술도 크게 발전했으며

In [ ]:
print(results.message['content'][0]['text'])

# 최신 AI Agent와 Strand Agents 상세 보고서

## 1. AI 에이전트(Agent)의 개념과 배경

AI 에이전트는 사용자의 목표를 달성하기 위해 데이터를 분석하고, 스스로 판단하며, 실제 행동을 수행하는 자율적 인공지능 시스템입니다. 기존의 생성형 AI가 콘텐츠 생성에 집중했다면, AI 에이전트는 문제 해결과 작업 실행까지 포괄합니다. 

**핵심 특징:**
- **자율성**: 인간의 개입 없이도 작업을 완료할 수 있는 능력
- **추론 및 계획 능력**: 복잡한 문제에 대한 단계별 접근 가능
- **도구 사용 능력**: 다양한 외부 도구와 API를 활용하여 작업 수행

최근 몇 년간 LLM(대규모 언어 모델)의 발전으로 인해 AI 에이전트 기술도 크게 발전했으며, 특히 ReAct(Reasoning and Acting) 논문 이후로 LLM이 추론하고, 계획하며, 환경에서 행동을 취할 수 있게 되었습니다.

## 2. Strands Agents 소개

Strands Agents는 AWS에서 출시한 오픈 소스 AI Agent SDK로, 모델 중심(model-driven) 접근 방식을 사용해 코드 몇 줄만으로 AI 에이전트를 구축하고 실행할 수 있게 해줍니다. 이미 Amazon Q Developer, AWS Glue, VPC Reachability Analyzer 등 여러 AWS 내부 팀에서 Strands Agents를 프로덕션에 사용하고 있습니다.

### 핵심 구성 요소

Strands Agents에서 AI 에이전트는 세 가지 핵심 요소로 구성됩니다:

1. **모델(Model)**: 에이전트의 '두뇌' 역할을 하는 LLM
   - Amazon Bedrock, Anthropic, Ollama, Meta 등 다양한 제공업체 모델 지원
   - 추론 및 도구 사용 기능을 갖춘 모델 활용

2. **도구(Tools)**: 에이전트가 작업을 수행하기 위해 사용하는 기능들
   - Model Context Protocol(MCP) 서버를 통해 수천 